In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# sensor_to_phoneme_ctc_fixed.py
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from collections import defaultdict
import random
import os

# -----------------------
# 0. CONFIG
# -----------------------
MAX_SEQUENCE_LENGTH = 400
RAW_FEATURE_DIM = 5
FEATURE_DIM = 7  # 5 raw + 2 ratios
NUM_PHONEMES = 40
BLANK_INDEX = 39
BATCH_SIZE = 16
NUM_EPOCHS = 50
LEARNING_RATE = 3e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

# -----------------------
# 1. ARPAbet
# -----------------------
arpabet = {
    "AA":0, "AE":1, "AH":2, "AO":3, "AW":4, "AY":5, "B":6, "CH":7, "D":8, "DH":9,
    "EH":10, "ER":11, "EY":12, "F":13, "G":14, "HH":15, "IH":16, "IY":17, "JH":18,
    "K":19, "L":20, "M":21, "N":22, "NG":23, "OW":24, "OY":25, "P":26, "R":27,
    "S":28, "SH":29, "T":30, "TH":31, "UH":32, "UW":33, "V":34, "W":35, "Y":36,
    "Z":37, "ZH":38, "<BLANK>":39
}
index_to_ph = {v: k for k, v in arpabet.items()}

# -----------------------
# 2. SENTENCES + PHONEMES
# -----------------------
SENTENCES = [
    "WHAT IS YOUR NAME", "WHERE ARE YOU FROM", "HOW ARE YOU", "NICE TO MEET YOU",
    "CAN YOU HELP ME", "THANK YOU VERY MUCH", "GOOD MORNING", "GOOD NIGHT",
    "SEE YOU LATER", "I AM FINE", "I LOVE YOU", "PLEASE WAIT", "EXCUSE ME",
    "SORRY ABOUT THAT", "I DON'T KNOW", "CAN YOU REPEAT", "TURN LEFT", "TURN RIGHT",
    "GO STRAIGHT", "STOP HERE", "I NEED WATER", "I AM HUNGRY", "I AM TIRED",
    "CALL THE DOCTOR", "OPEN THE DOOR", "CLOSE THE WINDOW", "TURN ON THE LIGHT",
    "TURN OFF THE FAN", "PLAY MUSIC", "STOP THE MUSIC", "WHAT TIME IS IT",
    "TODAY IS MONDAY", "TOMORROW IS TUESDAY", "I WANT COFFEE", "THIS IS DELICIOUS",
    "HOW MUCH IS THIS", "I WILL PAY", "KEEP THE CHANGE", "SEE YOU TOMORROW",
    "HAVE A GOOD DAY", "TAKE CARE", "BE CAREFUL", "DON'T WORRY", "I UNDERSTAND",
    "I DON'T UNDERSTAND", "SPEAK SLOWLY", "WRITE IT DOWN", "SHOW ME AGAIN",
    "ONE MORE TIME", "YES PLEASE", "NO THANKS"
]

SENTENCE_PHONEMES = {
    "WHAT IS YOUR NAME": ["W","AH","T","IH","Z","Y","AO","R","N","EY","M"],
    "WHERE ARE YOU FROM": ["W","EH","R","AA","R","Y","UW","F","R","AH","M"],
    "HOW ARE YOU": ["HH","AW","AA","R","Y","UW"],
    "NICE TO MEET YOU": ["N","AY","S","T","UW","M","IY","T","Y","UW"],
    "CAN YOU HELP ME": ["K","AE","N","Y","UW","HH","EH","L","P","M","IY"],
    "THANK YOU VERY MUCH": ["TH","AE","NG","K","Y","UW","V","EH","R","IY","M","AH","CH"],
    "GOOD MORNING": ["G","UH","D","M","AO","R","N","IH","NG"],
    "GOOD NIGHT": ["G","UH","D","N","AY","T"],
    "SEE YOU LATER": ["S","IY","Y","UW","L","EY","T","ER"],
    "I AM FINE": ["AY","AE","M","F","AY","N"],
    "I LOVE YOU": ["AY","L","AH","V","Y","UW"],
    "PLEASE WAIT": ["P","L","IY","Z","W","EY","T"],
    "EXCUSE ME": ["IH","K","S","K","Y","UW","Z","M","IY"],
    "SORRY ABOUT THAT": ["S","AO","R","IY","AH","B","AW","T","DH","AE","T"],
    "I DON'T KNOW": ["AY","D","OW","N","T","N","OW"],
    "CAN YOU REPEAT": ["K","AE","N","Y","UW","R","IH","P","IY","T"],
    "TURN LEFT": ["T","ER","N","L","EH","F","T"],
    "TURN RIGHT": ["T","ER","N","R","AY","T"],
    "GO STRAIGHT": ["G","OW","S","T","R","EY","T"],
    "STOP HERE": ["S","T","AA","P","HH","IY","R"],
    "I NEED WATER": ["AY","N","IY","D","W","AO","T","ER"],
    "I AM HUNGRY": ["AY","AE","M","HH","AH","NG","G","R","IY"],
    "I AM TIRED": ["AY","AE","M","T","AY","ER","D"],
    "CALL THE DOCTOR": ["K","AO","L","DH","AH","D","AA","K","T","ER"],
    "OPEN THE DOOR": ["OW","P","AH","N","DH","AH","D","AO","R"],
    "CLOSE THE WINDOW": ["K","L","OW","Z","DH","AH","W","IH","N","D","OW"],
    "TURN ON THE LIGHT": ["T","ER","N","AA","N","DH","AH","L","AY","T"],
    "TURN OFF THE FAN": ["T","ER","N","AO","F","DH","AH","F","AE","N"],
    "PLAY MUSIC": ["P","L","EY","M","Y","UW","Z","IH","K"],
    "STOP THE MUSIC": ["S","T","AA","P","DH","AH","M","Y","UW","Z","IH","K"],
    "WHAT TIME IS IT": ["W","AH","T","T","AY","M","IH","Z","IH","T"],
    "TODAY IS MONDAY": ["T","UW","D","EY","IH","Z","M","AH","N","D","EY"],
    "TOMORROW IS TUESDAY": ["T","AH","M","AA","R","OW","IH","Z","T","UW","Z","D","EY"],
    "I WANT COFFEE": ["AY","W","AA","N","T","K","AO","F","IY"],
    "THIS IS DELICIOUS": ["DH","IH","S","IH","Z","D","IH","L","IH","SH","AH","S"],
    "HOW MUCH IS THIS": ["HH","AW","M","AH","CH","IH","Z","DH","IH","S"],
    "I WILL PAY": ["AY","W","IH","L","P","EY"],
    "KEEP THE CHANGE": ["K","IY","P","DH","AH","CH","EY","N","JH"],
    "SEE YOU TOMORROW": ["S","IY","Y","UW","T","AH","M","AA","R","OW"],
    "HAVE A GOOD DAY": ["HH","AE","V","AH","G","UH","D","D","EY"],
    "TAKE CARE": ["T","EY","K","K","EH","R"],
    "BE CAREFUL": ["B","IY","K","EH","R","F","AH","L"],
    "DON'T WORRY": ["D","OW","N","T","W","ER","IY"],
    "I UNDERSTAND": ["AY","AH","N","D","ER","S","T","AE","N","D"],
    "I DON'T UNDERSTAND": ["AY","D","OW","N","T","AH","N","D","ER","S","T","AE","N","D"],
    "SPEAK SLOWLY": ["S","P","IY","K","S","L","OW","L","IY"],
    "WRITE IT DOWN": ["R","AY","T","IH","T","D","AW","N"],
    "SHOW ME AGAIN": ["SH","OW","M","IY","AH","G","EH","N"],
    "ONE MORE TIME": ["W","AH","N","M","AO","R","T","AY","M"],
    "YES PLEASE": ["Y","EH","S","P","L","IY","Z"],
    "NO THANKS": ["N","OW","TH","AE","NG","K","S"],
}

# -----------------------
# 3. manner + generator (unchanged logic)
# -----------------------
def get_manner(idx):
    stops = [6,8,19,26,30]; affricates = [7,18]; frics = [13,14,28,29,34,37,38]
    nasals = [21,22,23]; liquids = [20,27]; glides = [15,35,36]
    if idx in stops: return 0
    if idx in affricates: return 1
    if idx in frics: return 2
    if idx in nasals: return 3
    if idx in liquids: return 4
    if idx in glides: return 5
    return 6

def smooth_envelope(n):
    x = np.linspace(0, np.pi, n)
    return 0.7 + 0.3 * (1 - np.cos(x)) / 2

def generate_sensor_sequence(sentence):
    phonemes = SENTENCE_PHONEMES[sentence]
    ids = [arpabet[p] for p in phonemes]
    seq = []

    for idx in ids:
        manner = get_manner(idx)
        bases = [
            [1.3, 0.9, 0.1, 0.2, 0.8], [0.9, 0.2, 0.9, 1.0, 0.4], [0.8, 0.2, 0.9, 0.9, 0.3],
            [0.5, 0.3, 1.0, 0.3, 0.2], [0.6, 0.7, 0.6, 0.4, 0.6], [0.4, 0.8, 0.4, 0.5, 0.7],
            [0.7, 0.5, 0.5, 0.8, 0.4]
        ]
        base = np.array(bases[manner])
        base += np.random.normal(0, 0.15, 5)

        duration = np.random.randint(4, 13)
        t_scale = np.random.uniform(0.9, 1.1)
        t = np.linspace(0, 2*np.pi * t_scale, duration)
        env = smooth_envelope(duration) * np.random.uniform(0.7, 1.3)

        f1 = 1.5 + 0.8*manner + np.random.uniform(-0.5, 0.5)
        f2 = 2.5 + 0.6*manner + np.random.uniform(-0.7, 0.7)

        seg = np.zeros((duration, RAW_FEATURE_DIM))
        seg[:, 0] = base[0] + np.random.normal(0, 0.08, duration)
        seg[:, 1] = base[1] + np.random.normal(0, 0.08, duration)
        seg[:, 2] = base[2] + np.random.normal(0, 0.08, duration)
        seg[:, 3] = base[3] * np.sin(t * f1) * env + np.random.normal(0, 0.05, duration)
        seg[:, 4] = base[4] * np.sin(t * f2) * env + np.random.normal(0, 0.05, duration)

        seq.append(seg)

        if len(seq) > 1:
            prev = seq[-2]; curr = seg
            blend = np.random.randint(1, 4)
            if blend < len(prev) and blend < len(curr):
                prev[-blend:] = np.random.uniform(0.4, 0.7) * prev[-blend:] + (1 - np.random.uniform(0.4, 0.7)) * curr[:blend]

    full = np.concatenate(seq, axis=0)
    full += np.random.normal(0, 0.1, full.shape)
    drift = 0.05 * np.sin(np.linspace(0, 2*np.pi, len(full)))
    full *= (1 + drift[:, np.newaxis])
    full = np.clip(full, -3, 3)

    L = min(len(full), MAX_SEQUENCE_LENGTH)
    return full[:L], np.array(ids), L

# -----------------------
# 4. Build dataset (7 features) and normalize
# -----------------------
print("Generating dataset (5000 utterances)...")
X_list, Y_list, L_list = [], [], []

for _ in range(5000):
    sent = random.choice(SENTENCES)
    X_raw, Y_ids, L = generate_sensor_sequence(sent)  # (T,5)
    airflow = X_raw[:, 0]
    accel = X_raw[:, 1] + 1e-6
    moisture = X_raw[:, 2]
    r1 = airflow / accel
    r2 = moisture / (airflow + 1e-6)
    X_full = np.concatenate([X_raw, r1[:,None], r2[:,None]], axis=1)  # (T,7)
    X_pad = np.pad(X_full, ((0, MAX_SEQUENCE_LENGTH - L), (0, 0)), 'constant')
    X_list.append(X_pad.astype(np.float32))
    Y_list.append(Y_ids.astype(np.int64))
    L_list.append(L)

# Convert to tensor for normalization
X_tensor = torch.tensor(np.stack(X_list, axis=0), dtype=torch.float32)  # (N, T, F)
input_lengths = torch.tensor(L_list, dtype=torch.long)

# compute mean/std using valid frames only
with torch.no_grad():
    mask = torch.arange(MAX_SEQUENCE_LENGTH)[None,:] < input_lengths[:,None]
    denom = mask.sum().item() + 1e-8
    mean = (X_tensor * mask.unsqueeze(2)).sum(dim=(0,1)) / denom
    std = torch.sqrt(((X_tensor - mean)**2 * mask.unsqueeze(2)).sum(dim=(0,1)) / denom + 1e-6)
    X_tensor = (X_tensor - mean) / std
    X_tensor[~mask] = 0.0

# Create dataset as list of tuples (tensor, target_array, original_length, target_len)
dataset = []
for i in range(len(X_list)):
    dataset.append((X_tensor[i], Y_list[i], L_list[i], len(Y_list[i])))

# -----------------------
# 5. Model (keeps your arch)
# -----------------------
class SensorToPhoneme(nn.Module):
    def __init__(self):
        super().__init__()
        self.proj = nn.Linear(FEATURE_DIM, 512)
        self.pool = nn.AvgPool1d(4, 4)   # reduces time dim by 4
        self.gru = nn.GRU(512, 512, 4, batch_first=True, dropout=0.3)
        self.out = nn.Sequential(nn.Linear(512, NUM_PHONEMES), nn.LogSoftmax(dim=2))

    def forward(self, x):
        # x: (B, T, F)
        x = torch.tanh(self.proj(x))    # (B, T, 512)
        x = x.permute(0, 2, 1)          # (B, 512, T)
        x = self.pool(x)                # (B, 512, T//4)
        x = x.permute(0, 2, 1)          # (B, T//4, 512)
        x, _ = self.gru(x)              # (B, T//4, 512)
        x = self.out(x)                 # (B, T//4, C)
        return x.transpose(0, 1)        # (T//4, B, C) -> CTCLoss expects (T, N, C)

model = SensorToPhoneme().to(device)
ctc = nn.CTCLoss(blank=BLANK_INDEX, zero_infinity=True)
opt = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# -----------------------
# 6. collate_fn & DataLoader
# -----------------------
def collate_fn(batch):
    """
    batch: list of tuples (X_tensor (T,F), Y_array (variable), L, target_len)
    Return:
      Xs: (B, MAX_SEQUENCE_LENGTH, F) tensor
      targets_batch: 1D long tensor (concatenated targets for this batch)
      input_lens: original lengths (before pooling)
      target_lens: lengths of each target sequence
    """
    Xs = torch.stack([item[0] for item in batch], dim=0)  # (B, T, F)
    input_lens = torch.tensor([item[2] for item in batch], dtype=torch.long)
    target_lens = torch.tensor([item[3] for item in batch], dtype=torch.long)
    targets_batch = torch.cat([torch.tensor(item[1], dtype=torch.long) for item in batch], dim=0)
    return Xs, targets_batch, input_lens, target_lens

loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, drop_last=False, num_workers=0)

# -----------------------
# 7. Greedy decoder (phoneme output)
# -----------------------
def greedy_decode(log_probs, input_lens_reduced):
    """
    log_probs: (T, N, C) tensor (log-probabilities)
    input_lens_reduced: tensor of reduced input lengths (after pooling) - length N
    Returns: list of lists of ARPAbet phoneme symbols
    """
    # get predictions (T, N)
    with torch.no_grad():
        preds = log_probs.argmax(dim=2)  # (T, N)
        preds = preds.transpose(0,1).cpu().numpy()  # (N, T)
    results = []
    for i, Lr in enumerate(input_lens_reduced.cpu().numpy()):
        seq = preds[i, :Lr].tolist()
        out = []
        prev = -1
        for s in seq:
            if s != prev and s != BLANK_INDEX:
                out.append(index_to_ph.get(int(s), "?"))
            prev = s
        results.append(out)
    return results

# -----------------------
# 8. TRAIN
# -----------------------
print("\nTRAINING...")
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0.0
    batches = 0
    for Xs, targets_batch, input_lens_batch, target_lens_batch in loader:
        Xs = Xs.to(device)                      # (B, T, F)
        targets_batch = targets_batch.to(device) # 1D
        # reduced lengths after AvgPool1d(4,4)
        input_lens_reduced = (input_lens_batch // 4).clamp(min=1).to(device)
        target_lens_batch = target_lens_batch.to(device)

        opt.zero_grad()
        logp = model(Xs)  # (T_reduced, B, C)
        try:
            loss = ctc(logp, targets_batch, input_lens_reduced, target_lens_batch)
        except Exception as e:
            print("CTC error:", e)
            continue

        if torch.isnan(loss) or torch.isinf(loss):
            print("Skipping batch due to NaN/Inf loss")
            continue
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()

        total_loss += loss.item()
        batches += 1

    avg_loss = total_loss / max(1, batches)
    print(f"Epoch {epoch+1:3d}/{NUM_EPOCHS}  avg_loss={avg_loss:.4f}")

    # quick sanity decode on one batch sample from loader (no gradients)
    model.eval()
    with torch.no_grad():
        Xs, targets_batch, input_lens_batch, target_lens_batch = next(iter(loader))
        Xs = Xs.to(device)
        input_lens_reduced = (input_lens_batch // 4).clamp(min=1).to(device)
        logp = model(Xs)
        preds = greedy_decode(logp, input_lens_reduced)
        # print first sample (pred vs ground truth)
        print("Sample predicted phonemes (first example):", preds[0][:60])
        # ground truth of first example (convert ids to symbols)
        # need to slice out the first target from concatenated targets_batch
        # we can reconstruct from dataset since we have batch items
        gt_symbols = [index_to_ph[int(x)] for x in dataset[0][1][:dataset[0][3]]]
        print("Example ground truth (first dataset item):", gt_symbols)
    model.train()

# -----------------------
# 9. Save model + metadata
# -----------------------
save_path = "sensor_phoneme_model_fixed.pth"
torch.save({
    'model_state': model.state_dict(),
    'mean': mean,
    'std': std,
    'arpabet': arpabet,
    'index_to_ph': index_to_ph
}, save_path)
print("Model + metadata saved to", save_path)

# -----------------------
# 10. Example inference: "WHAT IS YOUR NAME"
# -----------------------
print("\nINFERENCE EXAMPLE on generated 'WHAT IS YOUR NAME' utterance:")
model.eval()
with torch.no_grad():
    # generate fresh sample (sensor) for the sentence
    x_raw, y_ids, L = generate_sensor_sequence("WHAT IS YOUR NAME")
    airflow = x_raw[:, 0]
    accel = x_raw[:, 1] + 1e-6
    moisture = x_raw[:, 2]
    r1 = airflow / accel
    r2 = moisture / (airflow + 1e-6)
    x_full = np.concatenate([x_raw, r1[:,None], r2[:,None]], axis=1)
    L = min(len(x_full), MAX_SEQUENCE_LENGTH)
    x_pad = np.pad(x_full, ((0, MAX_SEQUENCE_LENGTH - L), (0,0)), 'constant').astype(np.float32)
    # normalize using training mean/std
    x_tensor = (torch.tensor(x_pad, dtype=torch.float32) - mean) / std
    x_tensor[torch.arange(MAX_SEQUENCE_LENGTH) >= L] = 0.0
    x_tensor = x_tensor.unsqueeze(0).to(device)  # (1, T, F)
    logp = model(x_tensor)                       # (T_reduced, 1, C)
    input_len_reduced = torch.tensor([max(L // 4, 1)], dtype=torch.long).to(device)
    #input_len_reduced = torch.tensor([(L // 4).clamp(min=1)]).to(device)
    decoded = greedy_decode(logp, input_len_reduced)[0]
    print("Predicted phonemes (ARPAbet):", decoded)
    print("Ground truth phonemes (ARPAbet):", [index_to_ph[int(i)] for i in y_ids])

# End of script


In [ ]:

# =====================================================
# CELL 2: Load trained model + Inference + Accuracy
# =====================================================

import torch, numpy as np, random

# --- CONFIG (must match training) ---
MAX_SEQUENCE_LENGTH = 400
RAW_FEATURE_DIM = 5
FEATURE_DIM = 7
NUM_PHONEMES = 40
BLANK_INDEX = 39
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- ARPAbet map ---
arpabet = {
    "AA":0,"AE":1,"AH":2,"AO":3,"AW":4,"AY":5,"B":6,"CH":7,"D":8,"DH":9,
    "EH":10,"ER":11,"EY":12,"F":13,"G":14,"HH":15,"IH":16,"IY":17,"JH":18,
    "K":19,"L":20,"M":21,"N":22,"NG":23,"OW":24,"OY":25,"P":26,"R":27,
    "S":28,"SH":29,"T":30,"TH":31,"UH":32,"UW":33,"V":34,"W":35,"Y":36,
    "Z":37,"ZH":38,"<BLANK>":39
}
index_to_ph = {v:k for k,v in arpabet.items()}

# --- Minimal sentence→phoneme map (for accuracy) ---
SENTENCE_PHONEMES = {
    "WHAT IS YOUR NAME": ["W","AH","T","IH","Z","Y","AO","R","N","EY","M"],
    "SEE YOU TOMORROW": ["S","IY","Y","UW","T","AH","M","AA","R","OW"],
    "HELLO HOW ARE YOU": ["HH","AH","L","OW","HH","AW","AA","R","Y","UW"],
    "OPEN THE DOOR": ["OW","P","AH","N","DH","AH","D","AO","R"],
}

# --- Model class (same arch as training) ---
class SensorToPhoneme(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.proj = torch.nn.Linear(FEATURE_DIM,512)
        self.pool = torch.nn.AvgPool1d(4,4)
        self.gru  = torch.nn.GRU(512,512,4,batch_first=True,dropout=0.3)
        self.out  = torch.nn.Sequential(torch.nn.Linear(512,NUM_PHONEMES),
                                        torch.nn.LogSoftmax(dim=2))
    def forward(self,x):
        x = torch.tanh(self.proj(x))
        x = x.permute(0,2,1)
        x = self.pool(x)
        x = x.permute(0,2,1)
        x,_ = self.gru(x)
        x = self.out(x)
        return x.transpose(0,1)

# --- Greedy decoder ---
def greedy_decode(log_probs, input_lens_reduced):
    preds = log_probs.argmax(dim=2).transpose(0,1).cpu().numpy()
    results=[]
    for i,Lr in enumerate(input_lens_reduced.cpu().numpy()):
        seq = preds[i,:Lr].tolist()
        out=[]; prev=-1
        for s in seq:
            if s!=prev and s!=BLANK_INDEX:
                out.append(index_to_ph.get(int(s),""))
            prev=s
        results.append(out)
    return results

# --- Helper: compute phoneme error rate (edit distance) ---
def phoneme_error_rate(pred, true):
    m,n = len(pred), len(true)
    dp = np.zeros((m+1,n+1),dtype=int)
    for i in range(m+1): dp[i,0]=i
    for j in range(n+1): dp[0,j]=j
    for i in range(1,m+1):
        for j in range(1,n+1):
            cost = 0 if pred[i-1]==true[j-1] else 1
            dp[i,j]=min(dp[i-1,j]+1, dp[i,j-1]+1, dp[i-1,j-1]+cost)
    return dp[m,n]/max(n,1)

# --- Simplified generator (for demo / unseen sentences) ---
def smooth_envelope(n):
    x = np.linspace(0,np.pi,n)
    return 0.7 + 0.3*(1-np.cos(x))/2
def get_manner(idx):
    stops=[6,8,19,26,30]; affricates=[7,18]; frics=[13,14,28,29,34,37,38]
    nasals=[21,22,23]; liquids=[20,27]; glides=[15,35,36]
    if idx in stops: return 0
    if idx in affricates: return 1
    if idx in frics: return 2
    if idx in nasals: return 3
    if idx in liquids: return 4
    if idx in glides: return 5
    return 6
def generate_sensor_sequence(sentence):
    ids = np.random.randint(0,38,size=random.randint(8,14))
    seq=[]
    for _ in ids:
        seg=np.random.normal(0,1,(random.randint(6,12),RAW_FEATURE_DIM))
        seq.append(seg)
    full=np.concatenate(seq,axis=0)
    L=min(len(full),MAX_SEQUENCE_LENGTH)
    return full[:L], np.array(ids), L

# --- Load trained model ---
checkpoint = torch.load("sensor_phoneme_model_fixed.pth", map_location=device)
model = SensorToPhoneme().to(device)
model.load_state_dict(checkpoint["model_state"])
mean = checkpoint["mean"].to(device)   # ✅ move to same device
std  = checkpoint["std"].to(device)    # ✅ move to same device
print("✅ Model loaded successfully.")

# --- Inference function with PER if possible ---
def infer_sentence(sentence_text):
    model.eval()
    with torch.no_grad():
        x_raw,y_ids,L = generate_sensor_sequence(sentence_text)
        airflow=x_raw[:,0]; accel=x_raw[:,1]+1e-6; moisture=x_raw[:,2]
        r1=airflow/accel; r2=moisture/(airflow+1e-6)
        x_full=np.concatenate([x_raw,r1[:,None],r2[:,None]],axis=1)
        L=min(len(x_full),MAX_SEQUENCE_LENGTH)
        x_pad=np.pad(x_full,((0,MAX_SEQUENCE_LENGTH-L),(0,0)),'constant').astype(np.float32)
        
        x_tensor = (torch.tensor(x_pad, device=device) - mean) / std

        x_tensor[torch.arange(MAX_SEQUENCE_LENGTH)>=L]=0
        x_tensor=x_tensor.unsqueeze(0).to(device)
        logp=model(x_tensor)
        Lr=torch.tensor([max(L//4,1)],dtype=torch.long).to(device)
        pred=greedy_decode(logp,Lr)[0]

        print(f"\n🧠 Sentence: {sentence_text}")
        print("Predicted phonemes:", pred)
        if sentence_text in SENTENCE_PHONEMES:
            true=SENTENCE_PHONEMES[sentence_text]
            per=phoneme_error_rate(pred,true)
            print("Ground truth:", true)
            print(f"Phoneme Error Rate (PER): {per:.2f}  →  Accuracy ≈ {(1-per)*100:.1f}%")

# --- Try some examples ---
infer_sentence("WHAT IS YOUR NAME")
infer_sentence("SEE YOU TOMORROW")
infer_sentence("HELLO HOW ARE YOU")
infer_sentence("OPEN THE DOOR")
infer_sentence("NEW RANDOM SENTENCE TEST")
infer_sentence("My name is kd")


In [ ]:
print(x_tensor.shape)
print(x_tensor)


In [ ]:
print(input_lengths.shape)
print(input_lengths)